In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import random
from IPython import display
import time
import pickle
from AGame import *
%matplotlib inline
float_formatter = lambda x: "%.4f" % x
np.set_printoptions(formatter={'float_kind':float_formatter})
pd.options.display.float_format = '{:20,.3f}'.format

In [2]:
def getP(C):
    P = C/np.sum(C,axis=2)[:,:,np.newaxis]
    return(P)

In [3]:
env = aGame()

In [4]:
V = np.zeros(env.observation_space.n)
C = np.zeros([env.action_space.n,env.observation_space.n,env.observation_space.n])+0.01
R = np.zeros([env.action_space.n,env.observation_space.n,env.observation_space.n])
Pi = np.random.randint(env.action_space.n,size = (env.observation_space.n))
P = getP(C)
# P = np.zeros([env.observation_space.n,env.observation_space.n,])+1/(env.observation_space.n)
# V0List = []

In [ ]:
# f = open('train.pickle','rb') 
# V, C, P, R, Pi = pickle.load(f)
# f.close()

In [13]:
episodeMax = 5000
gamma = 0.99
done = False
weight = 0.1

rList = []
avgRList = []
flag = True

rationality = 0.5

for episode in range(episodeMax):
    rationality += 0.01
    display.clear_output(wait=True)
    print('Episode ',episode+1) 
#     time.sleep(0.8)
    state = env.reset()
    done = False
#     V0List.append(V[0])
    while not done:
        statePre = state
        if np.random.rand()<min(rationality,0.8):
            a1 = Pi[statePre[0]]
            a2 = Pi[statePre[1]]
        else:
            a1 = np.random.randint(3)
            a2 = np.random.randint(3)
            
        state,reward,done,info = env.step(a1,a2)
        a1 = env.a1
        a2 = env.a2
#         display.clear_output(wait=True)
#         env.render()
#         time.sleep(0.3)
        R[a1,statePre[0],state[0]] =  R[a1,statePre[0],state[0]]*(1-weight) + reward[0]*weight
        R[a2,statePre[1],state[1]] =  R[a2,statePre[1],state[1]]*(1-weight) + reward[1]*weight
        C[a1,statePre[0],state[0]] += 0.5
        C[a2,statePre[1],state[1]] += 0.5
        P = getP(C)
        
        # evaluate the current policy
        V[statePre[0]] = V[statePre[0]]*(1-weight)+(weight)*np.dot(P[a1,statePre[0],:],(R[a1,statePre[0],:]+V*gamma).T)
        V[statePre[1]] = V[statePre[1]]*(1-weight)+(weight)*np.dot(P[a2,statePre[1],:],(R[a2,statePre[1],:]+V*gamma).T)
        
        # improve policy
#         QBest = V
        Qsa =np.einsum('ijk,ijk->ji',P[:,:,:],R[:,:,:]+V[np.newaxis,np.newaxis,:]*gamma)
        Pi = np.argmax(Qsa+0.1*np.random.rand(env.observation_space.n,env.action_space.n),axis = 1)
        

#Saving the train result
f = open('train.pickle', 'wb')  
pickle.dump([V, C, P, R, Pi,rationality], f)
f.close()

Episode  5000


In [14]:
Pi.reshape(4,4)

array([[0, 0, 1, 1],
       [0, 0, 0, 0],
       [0, 0, 1, 0],
       [1, 1, 1, 1]])

In [18]:
V.reshape(4,4)

array([[4.2462, -2.5516, -4.3612, -4.6360],
       [8.2192, 4.2779, 3.1521, 2.8697],
       [10.1045, 4.6066, 4.1805, 3.3218],
       [10.4822, 5.2896, 4.3451, 4.0648]])

In [5]:
env.reset()

[0, 0]

In [11]:
env.step(2,2)
env.render()

|                                                              |
|                    XXXX A weird Game XXXX                    |
|                                                              |
|                         Robot DEFEND                         |
|                                                              |
|         ===========================================          |
|                                                              |
|                       Robot Ki: [*--]                        |
|                                                              |
|                     Xiaoli Hu Ki: [*--]                      |
|                                                              |
|         ===========================================          |
|                                                              |
|                       Xiaoli Hu DEFEND                       |
|                                                              |
|       [1]GATHER Ki  [2]